downloading datasets

In [1]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/noisy_test_data.txt

--2025-03-18 14:06:32--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375849 (367K) [text/plain]
Saving to: ‘train_data.txt’

train_data.txt      100%[===================>] 367.04K  --.-KB/s    in 0.04s   

2025-03-18 14:06:33 (8.58 MB/s) - ‘train_data.txt’ saved [375849/375849]

--2025-03-18 14:06:33--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reques

loading file

In [ ]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            sentence = []
            for token in line.strip().split():
                word, tag = token.rsplit('/', 1)  # Split word and tag
                sentence.append((word, tag))
            data.append(sentence)
    return data

# Load train and test data from files
train_data_file = '/content/train_data.txt'  # Path to your training data file
test_data_file = '/content/test_data.txt'    # Path to your test data file
noisy_test_data_file = '/content/noisy_test_data.txt'  # Path to your noisy test data file

train_data = load_data(train_data_file)
test_data = load_data(test_data_file)
noisy_test_data = load_data(noisy_test_data_file)

# Print a sample from the training data
print(train_data[0])

[('He', 'PRON'), ('let', 'VERB'), ('her', 'PRON'), ('tell', 'VERB'), ('him', 'PRON'), ('all', 'PRT'), ('about', 'ADP'), ('the', 'DET'), ('church', 'NOUN'), ('.', '.')]


In [ ]:
from collections import defaultdict

# Initialize count dictionaries
transition_counts = defaultdict(lambda: defaultdict(int))
emission_counts = defaultdict(lambda: defaultdict(int))
state_counts = defaultdict(int)
states = set()

# Count occurrences in training data
for sentence in train_data:
    prev_tag = "<s>"  # Start symbol
    for word, tag in sentence:
        states.add(tag)
        transition_counts[prev_tag][tag] += 1
        emission_counts[tag][word] += 1
        state_counts[tag] += 1
        prev_tag = tag
    transition_counts[prev_tag]["</s>"] += 1  # End symbol

states = list(states)

# Compute probabilities
def compute_probabilities(transition_counts, emission_counts, state_counts):
    transition_probs = defaultdict(lambda: defaultdict(float))
    emission_probs = defaultdict(lambda: defaultdict(float))

    # Compute transition probabilities
    for prev_state, next_states in transition_counts.items():
        total = sum(next_states.values())
        for next_state, count in next_states.items():
            transition_probs[prev_state][next_state] = count / total

    # Compute emission probabilities
    for state, words in emission_counts.items():
        total = state_counts[state]
        for word, count in words.items():
            emission_probs[state][word] = count / total

    return transition_probs, emission_probs

# Get probabilities
transition_probs, emission_probs = compute_probabilities(transition_counts, emission_counts, state_counts)

# Print sample probabilities
print(f"Sample transition probability: {list(transition_probs.items())[:1]}")
print(f"Sample emission probability: {list(emission_probs.items())[:1]}")


Sample transition probability: [('<s>', defaultdict(<class 'float'>, {'PRON': 0.149, 'NOUN': 0.1505, 'ADV': 0.0965, 'DET': 0.201, 'ADP': 0.1345, '.': 0.083, 'PRT': 0.031, 'VERB': 0.0485, 'ADJ': 0.0365, 'NUM': 0.016, 'CONJ': 0.053, 'X': 0.0005}))]
Sample emission probability: [('PRON', defaultdict(<class 'float'>, {'He': 0.06112759643916914, 'her': 0.024925816023738872, 'him': 0.04925816023738872, 'it': 0.13887240356083086, 'It': 0.035014836795252226, 'You': 0.011869436201780416, 'We': 0.01543026706231454, 'you': 0.05934718100890208, 'he': 0.13353115727002968, 'they': 0.05459940652818991, 'them': 0.037388724035608306, 'we': 0.03798219584569733, 'who': 0.052818991097922846, 'I': 0.09495548961424333, 'me': 0.02373887240356083, 'himself': 0.012462908011869436, 'She': 0.018991097922848664, 'yourself': 0.002967359050445104, 'she': 0.035014836795252226, "'im": 0.0005934718100890207, 'They': 0.013056379821958458, 'us': 0.013649851632047478, 'that': 0.037388724035608306, 'themselves': 0.0083086

In [ ]:
def viterbi_algorithm(sentence, states, transition_probs, emission_probs):
    V = [{}]  # Stores probabilities
    path = {}  # Stores the best path

    # Initialize base case (t=0)
    for state in states:
        V[0][state] = transition_probs["<s>"].get(state, 1e-6) * emission_probs[state].get(sentence[0], 1e-6)
        path[state] = [state]

    # Recursively compute probabilities for t > 0
    for t in range(1, len(sentence)):
        V.append({})
        new_path = {}

        for curr_state in states:
            max_prob, best_prev_state = max(
                (V[t - 1][prev_state] * transition_probs[prev_state].get(curr_state, 1e-6) *
                 emission_probs[curr_state].get(sentence[t], 1e-6), prev_state)
                for prev_state in states
            )
            V[t][curr_state] = max_prob
            new_path[curr_state] = path[best_prev_state] + [curr_state]

        path = new_path

    # Find the best final state
    max_prob, best_final_state = max((V[-1][state] * transition_probs[state].get("</s>", 1e-6), state) for state in states)
    return path[best_final_state]

# Example test sentence
test_sentence = ["The", "dog", "barks"]
predicted_tags = viterbi_algorithm(test_sentence, states, transition_probs, emission_probs)
print("Predicted POS Tags:", predicted_tags)


Predicted POS Tags: ['DET', 'NOUN', '.']


In [ ]:
# Function to decode noisy sentences
def decode_noisy_data(noisy_sentences, states, transition_probs, emission_probs):
    decoded_sentences = []
    for sentence in noisy_sentences:
        words = [word for word, _ in sentence]  # Extract words from noisy sentences
        predicted_tags = viterbi_algorithm(words, states, transition_probs, emission_probs)
        decoded_sentences.append(list(zip(words, predicted_tags)))
    return decoded_sentences

# Decode noisy test sentences
decoded_noisy_sentences = decode_noisy_data(noisy_test_data, states, transition_probs, emission_probs)

# Print some decoded sentences
for i in range(5):  # Display the first 5 results
    print(decoded_noisy_sentences[i])


[('Get', 'DET'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'ADP'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', '.'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]
[('By', 'ADP'), ('permitting', 'DET'), ('freshman', 'ADJ'), ('students', 'NOUN'), ('we', 'PRON'), ('might', 'VERB'), ('extend', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('for', 'ADP'), ('such', 'PRT'), ('a', 'DET'), ('course', 'NOUN'), ('to', 'ADP'), ('some', 'DET'), ('individuals', 'NOUN'), ('who', 'PRON'), ('otherwise', 'ADV'), ('might', 'VERB'), ('never', 'ADV'), ('get', 'VERB'), ('to', 'PRT'), ('take', 'VERB'), ('it', 'PRON'), ('.', '.')]
[('In', 'ADP'), ('stating', 'ADP'), ('this', 'DET'), ('position', 'NOUN'), (',', '.'), ('we', 'PRON'), ('should', 'VERB'), ('like', 'ADP'), ('to', 'PRT'), ('make', 'VERB'), ('it', 'PRON'), ('clear', 'ADJ'), ('to', 'ADP'), ('you', 'PRON'), ('that', 'ADP'), ('we', 'PRON'), ('cannot', 'VERB'), ('expect', 'VERB'), ('a

###Evaluation
An example of evaluation:

In [ ]:
sentence = [('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]
predicted_tags = ['DET', 'NOUN', 'CONJ', 'NOUN', 'ADP', 'PRON', 'VERB', 'NOUN', '.', 'CONJ', 'NOUN', 'ADJ', '.']
true_tags = ('VERB', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'CONJ', 'NOUN', 'ADJ', '.')
correct = 0
total = 0
correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
total += len(true_tags)
accuracy = correct / total

print(f"Baseline Viterbi Accuracy: {accuracy * 100:.2f}%")
# print(f"Viterbi with Noise Handling Accuracy: {accuracy * 100:.2f}%")  # similarly calculate for Noise Handling


Baseline Viterbi Accuracy: 76.92%


In [ ]:
# Function to evaluate accuracy
def evaluate_accuracy(test_data, states, transition_probs, emission_probs):
    correct = total = 0
    for sentence in test_data:
        words, true_tags = zip(*sentence)  # Extract words and their actual tags
        predicted_tags = viterbi_algorithm(words, states, transition_probs, emission_probs)
        correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
        total += len(true_tags)
    return correct / total

# Compute accuracy on test data
accuracy = evaluate_accuracy(test_data, states, transition_probs, emission_probs)
print(f"Baseline Viterbi Accuracy: {accuracy * 100:.2f}%")

# Compute accuracy on noisy test data
noisy_accuracy = evaluate_accuracy(noisy_test_data, states, transition_probs, emission_probs)
print(f"Noise-Handled Viterbi Accuracy: {noisy_accuracy * 100:.2f}%")


Baseline Viterbi Accuracy: 90.18%
Noise-Handled Viterbi Accuracy: 81.77%
